In [4]:
# import dataset
import pandas as pd

df = pd.read_parquet("hf://datasets/phosseini/multimodal_satire/data/train-00000-of-00001-593392955b371d90.parquet")
df.head()

,url,headline,image_link,is_satire
0,https://babylonbee.com/news/this-aint-it-chief...,"'This Ain’t It, Chief' Updated To 'This Ain’t ...",https://babylonbee.com/img/articles/article-65...,1
1,https://babylonbee.com/news/announcement-the-b...,Using Our Prophetic Powers For Good: The Babyl...,https://babylonbee.com/img/articles/article-65...,1
2,https://babylonbee.com/news/study-wearing-a-ma...,Study Finds Connection Between Wearing A Mask ...,https://babylonbee.com/img/articles/article-65...,1
3,https://babylonbee.com/news/benny-hinn-flies-p...,Benny Hinn Travels To Dagobah To Master His Po...,https://babylonbee.com/img/articles/article-65...,1
4,https://babylonbee.com/news/white-house-adds-c...,White House Adds Cry Room For Fussy Reporters,https://babylonbee.com/img/articles/article-65...,1


In [6]:
## Downloading images

# Define custom headers
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
output_folder = 'images'

# Download images with custom headers
for i, url in enumerate(df.image_link):
    try:
        # Define the image file name
        file_extension = url.split('.')[-1]  # Get the file extension
        file_name = f'image_{i}.{file_extension}'
        file_path = os.path.join(output_folder, file_name)

        # Check if the file already exists
        if os.path.exists(file_path):
            print(f"Skipped: {file_name} (already exists)")
            continue

        # Fetch the image with custom headers
        response = requests.get(url, headers=headers, timeout=10)  # Add headers to the request
        
        # Check if the request was successful
        if response.status_code == 200:
            # Save the image
            with open(file_path, 'wb') as f:
                f.write(response.content)
            print(f"Downloaded: {file_name}")
        else:
            print(f"Failed to download: {url} (status code: {response.status_code})")
    except Exception as e:
        print(f"Error downloading {url}: {e}")


Skipped: image_0.jpg (already exists)
Skipped: image_1.jpg (already exists)
Skipped: image_2.jpg (already exists)
Skipped: image_3.jpg (already exists)
Skipped: image_4.jpg (already exists)
Skipped: image_5.jpg (already exists)
Skipped: image_6.jpg (already exists)
Skipped: image_7.jpg (already exists)
Skipped: image_8.jpg (already exists)
Skipped: image_9.jpg (already exists)
Skipped: image_10.jpg (already exists)
Skipped: image_11.jpg (already exists)
Skipped: image_12.jpg (already exists)
Skipped: image_13.jpg (already exists)
Skipped: image_14.jpg (already exists)
Skipped: image_15.jpg (already exists)
Skipped: image_16.jpg (already exists)
Skipped: image_17.jpg (already exists)
Skipped: image_18.jpg (already exists)
Skipped: image_19.jpg (already exists)
Skipped: image_20.jpg (already exists)
Skipped: image_21.jpg (already exists)
Skipped: image_22.jpg (already exists)
Skipped: image_23.jpg (already exists)
Skipped: image_24.jpg (already exists)
Skipped: image_25.jpg (already exis

In [9]:
import os
import pandas as pd

# Assume `df` is your original DataFrame with 'headline' and 'is_satire'
X_headline, Y_is_satire = df['headline'], df['is_satire']

# Specify folder path
folder_path = 'images'

# Batch processing
batch_size = 1000  # Number of rows to process at once
results = []  # To store results incrementally

# Function to check if the image exists in any of the allowed formats
def get_image_path(idx, folder_path):
    for ext in ['.jpg', '.jpeg', '.png']:
        file_name = f'image_{idx}{ext}'  # Construct file name with different extensions
        file_path = os.path.join(folder_path, file_name)
        if os.path.exists(file_path):  # Check if the file exists
            return file_path
    return None  # Return None if no valid image is found

for start_idx in range(0, len(X_headline), batch_size):
    print(f"Processing batch {start_idx} to {start_idx + batch_size - 1}...")
    
    # Process one batch
    batch = []
    for idx in range(start_idx, min(start_idx + batch_size, len(X_headline))):
        file_path = get_image_path(idx, folder_path)  # Get the valid image path
        
        # Append the file path or None to batch
        batch.append((X_headline.iloc[idx], file_path, Y_is_satire.iloc[idx]))
    
    # Add batch to results
    results.extend(batch)
    
    # Save results periodically to a pickle file
    pd.DataFrame(results, columns=['headline', 'image_path', 'is_satire']).to_pickle('processed_data.pkl')

# Convert results to a DataFrame
final_df = pd.DataFrame(results, columns=['headline', 'image_path', 'is_satire'])

# Save final DataFrame
final_df.to_pickle('final_data.pkl')
print("Processing completed and data saved.")


Processing batch 0 to 999...
Processing batch 1000 to 1999...
Processing batch 2000 to 2999...
Processing batch 3000 to 3999...
Processing batch 4000 to 4999...
Processing batch 5000 to 5999...
Processing batch 6000 to 6999...
Processing batch 7000 to 7999...
Processing batch 8000 to 8999...
Processing batch 9000 to 9999...
Processing completed and data saved.


In [10]:
# Load the DataFrame (if not already in memory)
final_df = pd.read_pickle('final_data.pkl')

# Remove rows where 'image_path' is None
filtered_df = final_df[final_df['image_path'].notna()]

# Optionally reset the index
filtered_df = filtered_df.reset_index(drop=True)

# Save the filtered DataFrame if needed
filtered_df.to_pickle('filtered_data.pkl')

# Display the first few rows
print(filtered_df.head())


                                            headline          image_path  \
0  'This Ain’t It, Chief' Updated To 'This Ain’t ...  images/image_0.jpg   
1  Using Our Prophetic Powers For Good: The Babyl...  images/image_1.jpg   
2  Study Finds Connection Between Wearing A Mask ...  images/image_2.jpg   
3  Benny Hinn Travels To Dagobah To Master His Po...  images/image_3.jpg   
4      White House Adds Cry Room For Fussy Reporters  images/image_4.jpg   

   is_satire  
0          1  
1          1  
2          1  
3          1  
4          1  
